# List of Exposures with Hologram at USDF


- work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2022/10/31



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_39/summit_utils/python', '/home/d/dagoret/repos/repos_w_2022_39/drp_pipe/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_tools/gb9b489fd42+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_drp/g12d5fbe6a5+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/faro/gbb36616a4b+5e2d81108f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcmcal/g5147cf65fe+deeae05922/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcm/g3dbc634184/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/jointcal/ge7491f621d+ceb865f59e/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_cfht/gd4e7b81488+e2797d1921/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
repo = "/sdf/group/rubin/repo/main"
#repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [9]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022062800004,AT_O_20220628_000004,20220628,4,bias,Park position,SDSSr~empty,NaN,0.0,NaN,NaN,NaN
1,2022070600001,AT_O_20220706_000001,20220706,1,test,UNKNOWN,unknown~unknown,NaN,5.0,NaN,NaN,NaN
2,2022070700001,AT_O_20220707_000001,20220707,1,bias,UNKNOWN,empty~ronchi170lpmm,NaN,0.0,NaN,NaN,NaN
3,2022071100001,AT_O_20220711_000001,20220711,1,bias,UNKNOWN,FELH0600~holo4_003,NaN,0.0,NaN,NaN,NaN
4,2022071200001,AT_O_20220712_000001,20220712,1,bias,UNKNOWN,SDSSg~holo4_003,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44232,2022060900935,AT_O_20220609_000935,20220609,935,cwfs,HD 221054,SDSSg~empty,21.707667,30.0,352.285362,-51.019608,269.453648
44233,2022060900936,AT_O_20220609_000936,20220609,936,cwfs,HD 221054,SDSSg~empty,21.670582,30.0,352.28535,-51.019575,269.453634
44234,2022060900937,AT_O_20220609_000937,20220609,937,cwfs,HD 221054,SDSSg~empty,21.594164,30.0,352.285373,-51.019592,269.453635
44235,2022060900938,AT_O_20220609_000938,20220609,938,cwfs,HD 221054,SDSSg~empty,21.559853,30.0,352.285369,-51.019605,269.453759


## Selection of science

In [12]:
df_science = df_exposure[df_exposure.type == 'science']

In [13]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_9145/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [14]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
2065,2022063000741,AT_O_20220630_000741,20220630,741,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.216657,30.0,309.341775,-5.594321,50.470185
2066,2022063000742,AT_O_20220630_000742,20220630,742,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.311557,30.0,309.341675,-5.594353,50.472142
2067,2022063000743,AT_O_20220630_000743,20220630,743,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.36748,30.0,309.436196,-5.594359,50.230394
2068,2022063000744,AT_O_20220630_000744,20220630,744,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.459853,30.0,309.436102,-5.594346,50.231639
2069,2022063000745,AT_O_20220630_000745,20220630,745,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.571891,30.0,328.698147,17.412737,83.126668
2070,2022063000746,AT_O_20220630_000746,20220630,746,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.597445,30.0,328.698026,17.412692,83.124157
2071,2022063000747,AT_O_20220630_000747,20220630,747,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.759167,30.0,328.50127,17.506591,82.501275
2072,2022063000748,AT_O_20220630_000748,20220630,748,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.786121,30.0,328.501103,17.506714,82.502325
2073,2022063000749,AT_O_20220630_000749,20220630,749,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.801233,30.0,328.599795,17.506644,82.299976
2074,2022063000750,AT_O_20220630_000750,20220630,750,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.82869,30.0,328.599615,17.506728,82.298997


In [15]:
df_science.day_obs.unique()

array([20200127, 20200128, 20200129, 20200212, 20200217, 20200218,
       20200219, 20200220, 20200221, 20200312, 20200313, 20200314,
       20200315, 20200316, 20210119, 20210120, 20210121, 20210126,
       20210209, 20210216, 20210217, 20210218, 20210309, 20210310,
       20210311, 20210323, 20210524, 20210525, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210727, 20210804,
       20210817, 20210907, 20210908, 20210909, 20211005, 20211006,
       20211102, 20211103, 20211104, 20211130, 20220208, 20220209,
       20220215, 20220216, 20220217, 20220308, 20220311, 20220316,
       20220317, 20220405, 20220406, 20220407, 20220503, 20220504,
       20220505, 20220524, 20220525, 20220526, 20220607, 20220608,
       20220609, 20220628, 20220629, 20220630])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [16]:
date_sel = 20211103

In [17]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [18]:
len(df_science_selected)

263

# List of filters

In [19]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['FELH0600~empty', 'SDSSg~empty', 'FELH0600~holo4_003',
       'empty~holo4_003', 'BG40~holo4_003', 'SDSSg~holo4_003'],
      dtype=object)

In [20]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['FELH0600~holo4_003', 'empty~holo4_003', 'BG40~holo4_003',
       'SDSSg~holo4_003'], dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [21]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

FELH0600~holo4_003
visitdispersers_20211103_filt_FELH0600-holo4_003.list
empty~holo4_003
visitdispersers_20211103_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20211103_filt_BG40-holo4_003.list
SDSSg~holo4_003
visitdispersers_20211103_filt_SDSSg-holo4_003.list


/tmp/ipykernel_9145/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_9145/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_9145/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [24]:
len(all_def_science_selected_final)

4

In [25]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
25236,2021110300448,AT_O_20211103_000448,20211103,448,science,HD 37795,FELH0600~holo4_003,30.04624,2.0,84.912487,-34.074126,357.466664
25237,2021110300449,AT_O_20211103_000449,20211103,449,science,HD 37795,FELH0600~holo4_003,29.922213,2.0,84.912307,-33.991074,357.466751
25238,2021110300450,AT_O_20211103_000450,20211103,450,science,HD 37795,FELH0600~holo4_003,29.759992,2.0,84.858917,-33.996452,357.467396
25239,2021110300451,AT_O_20211103_000451,20211103,451,science,HD 37795,FELH0600~holo4_003,29.208396,1.0,84.912423,-34.074294,357.966363
25240,2021110300452,AT_O_20211103_000452,20211103,452,science,HD 37795,FELH0600~holo4_003,29.095972,1.0,84.925386,-34.039079,357.965341
25241,2021110300453,AT_O_20211103_000453,20211103,453,science,HD 37795,FELH0600~holo4_003,28.980319,1.0,84.923574,-34.038003,357.966044
25243,2021110300455,AT_O_20211103_000455,20211103,455,science,HD 38666,FELH0600~holo4_003,29.505123,1.0,86.499726,-32.306347,354.723354
25244,2021110300456,AT_O_20211103_000456,20211103,456,science,HD 38666,FELH0600~holo4_003,29.375247,1.0,86.508605,-32.271477,354.725128
25250,2021110300462,AT_O_20211103_000462,20211103,462,science,HD 38666,FELH0600~holo4_003,27.759744,30.0,86.508595,-32.270625,354.723764
25257,2021110300468,AT_O_20211103_000468,20211103,468,science,HD 38666,FELH0600~holo4_003,26.900151,30.0,86.508557,-32.270622,354.723659


In [26]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
25247,2021110300459,AT_O_20211103_000459,20211103,459,science,HD 38666,empty~holo4_003,28.24208,30.0,86.508515,-32.270641,354.723745
25254,2021110300465,AT_O_20211103_000465,20211103,465,science,HD 38666,empty~holo4_003,27.382811,30.0,86.50853,-32.270625,354.723849
25260,2021110300471,AT_O_20211103_000471,20211103,471,science,HD 38666,empty~holo4_003,26.526583,30.0,86.508552,-32.270615,354.723461
25266,2021110300477,AT_O_20211103_000477,20211103,477,science,HD 38666,empty~holo4_003,25.652265,30.0,86.508558,-32.27063,354.724046
25272,2021110300483,AT_O_20211103_000483,20211103,483,science,HD 38666,empty~holo4_003,24.795164,30.0,86.508554,-32.270624,354.724115
25278,2021110300489,AT_O_20211103_000489,20211103,489,science,HD 38666,empty~holo4_003,23.93414,30.0,86.508511,-32.270625,354.723861
25284,2021110300495,AT_O_20211103_000495,20211103,495,science,HD 38666,empty~holo4_003,23.055235,30.0,86.508571,-32.270593,354.722389
25290,2021110300501,AT_O_20211103_000501,20211103,501,science,HD 38666,empty~holo4_003,22.19714,30.0,86.508523,-32.270605,354.723577
25296,2021110300507,AT_O_20211103_000507,20211103,507,science,HD 38666,empty~holo4_003,21.338854,30.0,86.508493,-32.27062,354.723602
25302,2021110300513,AT_O_20211103_000513,20211103,513,science,HD 38666,empty~holo4_003,20.378,30.0,86.508515,-32.270743,354.723787


In [27]:
all_def_science_selected_final[2]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
25248,2021110300460,AT_O_20211103_000460,20211103,460,science,HD 38666,BG40~holo4_003,28.080096,30.0,86.508482,-32.270632,354.723756
25255,2021110300466,AT_O_20211103_000466,20211103,466,science,HD 38666,BG40~holo4_003,27.220839,30.0,86.508624,-32.270623,354.72379
25261,2021110300472,AT_O_20211103_000472,20211103,472,science,HD 38666,BG40~holo4_003,26.364422,30.0,86.508525,-32.270609,354.723692
25267,2021110300478,AT_O_20211103_000478,20211103,478,science,HD 38666,BG40~holo4_003,25.490008,30.0,86.50851,-32.270621,354.723686
25273,2021110300484,AT_O_20211103_000484,20211103,484,science,HD 38666,BG40~holo4_003,24.632855,30.0,86.508533,-32.270619,354.723792
25279,2021110300490,AT_O_20211103_000490,20211103,490,science,HD 38666,BG40~holo4_003,23.771771,30.0,86.508496,-32.270626,354.723813
25285,2021110300496,AT_O_20211103_000496,20211103,496,science,HD 38666,BG40~holo4_003,22.892863,30.0,86.508616,-32.270615,354.723616
25291,2021110300502,AT_O_20211103_000502,20211103,502,science,HD 38666,BG40~holo4_003,22.034692,30.0,86.50853,-32.27062,354.723527
25297,2021110300508,AT_O_20211103_000508,20211103,508,science,HD 38666,BG40~holo4_003,21.17642,30.0,86.508559,-32.270625,354.723398
25303,2021110300514,AT_O_20211103_000514,20211103,514,science,HD 38666,BG40~holo4_003,20.211981,30.0,86.508544,-32.270611,354.723377


In [28]:
all_def_science_selected_final[3]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
25249,2021110300461,AT_O_20211103_000461,20211103,461,science,HD 38666,SDSSg~holo4_003,27.921729,30.0,86.508584,-32.270606,354.723684
25256,2021110300467,AT_O_20211103_000467,20211103,467,science,HD 38666,SDSSg~holo4_003,27.062237,30.0,86.508558,-32.270624,354.723616
25262,2021110300473,AT_O_20211103_000473,20211103,473,science,HD 38666,SDSSg~holo4_003,26.205784,30.0,86.508515,-32.270622,354.723793
25268,2021110300479,AT_O_20211103_000479,20211103,479,science,HD 38666,SDSSg~holo4_003,25.331445,30.0,86.508679,-32.270614,354.723854
25274,2021110300485,AT_O_20211103_000485,20211103,485,science,HD 38666,SDSSg~holo4_003,24.474113,30.0,86.508572,-32.270626,354.723817
25280,2021110300491,AT_O_20211103_000491,20211103,491,science,HD 38666,SDSSg~holo4_003,23.612994,30.0,86.508566,-32.27062,354.723774
25286,2021110300497,AT_O_20211103_000497,20211103,497,science,HD 38666,SDSSg~holo4_003,22.737461,30.0,86.508562,-32.270616,354.723577
25292,2021110300503,AT_O_20211103_000503,20211103,503,science,HD 38666,SDSSg~holo4_003,21.875809,30.0,86.508585,-32.270614,354.723782
25298,2021110300509,AT_O_20211103_000509,20211103,509,science,HD 38666,SDSSg~holo4_003,21.017505,30.0,86.508613,-32.270612,354.723513
25304,2021110300515,AT_O_20211103_000515,20211103,515,science,HD 38666,SDSSg~holo4_003,20.056521,30.0,86.508532,-32.270634,354.723831


In [29]:
all_def_science_selected_final[4]

IndexError: list index out of range